In [1]:
import pandas as pd
import numpy as np

# Read in observations

In [2]:
import os

pos_path = 'data/review_polarity/txt_sentoken/pos/'
neg_path = 'data/review_polarity/txt_sentoken/neg/'
pos_rev = []
neg_rev = []

# assign positive reviews to list
for filename in os.listdir(pos_path):
    with open(pos_path + filename, 'r') as f:
        pos_rev.append(f.read())

# assign negative reviews to list
for filename in os.listdir(neg_path):
    with open(neg_path + filename, 'r') as f:
        neg_rev.append(f.read())

In [3]:
# create dictionary with pos/neg labels
data_pos = {'review': pos_rev, 'rating': 'positive'}
data_neg = {'review': neg_rev, 'rating': 'negative'}

In [4]:
df = pd.DataFrame(data_pos)
df_rev = pd.concat([df, pd.DataFrame(data_neg)])

In [5]:
df_rev.head()

,review,rating
0,""" love is the devil "" is a challenging film ,...",positive
1,"in some respects , rush hour is the ultimate e...",positive
2,"martin scorsese's kundun , which chronicles ro...",positive
3,expectation rating : a bit worse than expected...,positive
4,note : some may consider portions of the follo...,positive


In [6]:
X = df_rev.iloc[:, 0]

In [7]:
X.head()

0     " love is the devil " is a challenging film ,...
1    in some respects , rush hour is the ultimate e...
2    martin scorsese's kundun , which chronicles ro...
3    expectation rating : a bit worse than expected...
4    note : some may consider portions of the follo...
Name: review, dtype: object

In [8]:
y = df_rev.iloc[:, 1]

In [9]:
y.head()

0    positive
1    positive
2    positive
3    positive
4    positive
Name: rating, dtype: object

## Test/train split

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
X_train.head()

968    a big surprise to me . \nthe good trailer had ...
240    well i'll be damned . . . \nthe canadians can ...
819     * * * * * * minor plot spoilers in review * *...
692    at first glance , it appears that the home alo...
420    note : some may consider portions of the follo...
Name: review, dtype: object

# Sklearn Solution

## Create Sparse Matrix

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
vectorizer = CountVectorizer()

In [13]:
vectorizer.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [14]:
feature_names = vectorizer.get_feature_names()

In [15]:
len(feature_names)

36291

In [16]:
X_train_vec = vectorizer.transform(X_train)

In [17]:
vect_view=pd.DataFrame(X_train_vec.toarray(),columns=feature_names)
vect_view.head()

,00,000,0009f,007,03,04,05,05425,10,100,...,zuehlke,zuko,zukovsky,zundel,zurg,zus,zwick,zwigoff,zycie,zzzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
vect_view.shape

(1600, 36291)

## Modeling

In [39]:
from sklearn import naive_bayes
from sklearn.metrics import accuracy_score, classification_report

In [20]:
X_test_vec = vectorizer.transform(X_test)

In [21]:
model = naive_bayes.MultinomialNB()
model.fit(X_train_vec, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
print ("Accuracy: %.3f"% accuracy_score(y_test, model.predict(X_test_vec)))
print (classification_report(y_test, model.predict(X_test_vec)))

Accuracy: 0.840
             precision    recall  f1-score   support

   negative       0.83      0.85      0.84       201
   positive       0.85      0.83      0.84       199

avg / total       0.84      0.84      0.84       400



# Manual Solution

In [13]:
import nltk

# Functions

In [14]:
# create sparse matrix of words
# input is dataframe of text

def create_word_mat(df_docs_text, df_docs_labels):
    # initialize empty list to store counts for each observation
    word_dicts = []
    
    for doc in df_docs_text:
        word_counts = {}
        # split sentences into individual words
        words = nltk.word_tokenize(doc)
        # calc word counts for each word in observation
        for word in words:
            if word.isalnum():
                if word in word_counts.values():
                    word_counts[word] += 1
                else:
                    word_counts[word] = 1
            else:
                pass
        # add word counts for observation to list
        word_dicts.append(word_counts)
    
    # create dataframe of all observation word counts (0's fill empty word count cells)
    df_words = pd.DataFrame.from_records(word_dicts).fillna(0)
    # add class labels to dataframe
    df_words['class_labels'] = df_docs_labels
    return(df_words)

In [15]:
# calculate probabilities of words for each class
# input is word sparse matrix including last column of class labels

def calc_probas(df_words):
    # word counts by class
    word_freq = df_words.groupby(df_words.columns[-1]).sum()
    # total number of words in sample (to add for smoothing of zero values)
    word_num = len(word_freq.columns)
    # calc word totals per class
    word_freq['class_total'] = word_freq.sum(axis=1) + word_num
    # calc probabilities of each word appearing in each class
    word_probas = (word_freq.iloc[:, 0:-1] + 1).div(word_freq['class_total'], axis=0)
    return(word_probas)

In [23]:
# calculate probabilities of words for each class
# input is word sparse matrix including last column of class labels

def calc_probas(df_words):
    # word counts by class
    word_freq = df_words.groupby(df_words.columns[-1]).sum()
    # total number of words in sample (to add for smoothing of zero values)
    word_num = len(word_freq.columns)
    # calc word totals per class
    word_freq['class_total'] = word_freq.sum(axis=1) + word_num
    # calc probabilities of each word appearing in each class
    word_probas = np.log((word_freq.iloc[:, 0:-1] + 1).div(word_freq['class_total'], axis=0))
    return(word_probas)

In [30]:
# predict class for given statement
# input is statement to predict and word probability dataframe

def pred_class(statement, df_probas):
    # break statement into individual words
    words = nltk.word_tokenize(statement)
    # remove words not in training set
    words = set(words).intersection(df_probas.columns)
    # calculate probabilities of statement per class
    probas = df_probas[list(words)].prod(axis=1)
    # return predicted classes
    return(probas.idxmin(axis=1))

In [17]:
# need to calc TP for all classes and average for prec, recall, f1

def metrics(prediction, actual):
    
    classes = prediction.append(actual).unique()
    
    TP = sum((prediction == classes[0]) & (prediction == actual))
    TN = sum((prediction == classes[1]) & (prediction == actual))
    FN = sum((prediction == classes[1]) & (prediction != actual))
    FP = sum((prediction == classes[0]) & (prediction != actual))
    
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * (recall * precision) / (recall + precision)
    
    print("Scores")
    print("-"*15)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1:", f1)

## Count word frequency

In [18]:
df_words = create_word_mat(X_train.reset_index(drop=True), y_train.reset_index(drop=True))
df_words.head()

,0,00,000,0009f,007,03,04,05,05425,1,...,zucker,zuehlke,zuko,zukovsky,zundel,zurg,zwick,zwigoff,zycie,class_labels
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,positive
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,positive
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,positive
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,positive
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,positive


In [24]:
df_probas = calc_probas(df_words)
df_probas

,0,00,000,0009f,007,03,04,05,05425,1,...,zsigmond,zucker,zuehlke,zuko,zukovsky,zundel,zurg,zwick,zwigoff,zycie
class_labels,,,,,,,,,,,,,,,,,,,,,
negative,-9.783176,-11.457152,-9.029404,-12.555764,-11.169470,-11.862617,-11.862617,-11.457152,-11.862617,-8.251699,...,-11.862617,-10.946326,-12.555764,-12.555764,-12.555764,-12.555764,-12.555764,-11.862617,-11.862617,-12.555764
positive,-10.139916,-11.526210,-8.935943,-11.931675,-10.833063,-12.624822,-12.624822,-11.931675,-12.624822,-8.465939,...,-12.624822,-11.526210,-11.931675,-11.931675,-11.931675,-11.931675,-11.931675,-10.833063,-12.624822,-11.931675


In [31]:
y_predict = X_test.apply(lambda x: pred_class(x, df_probas))

In [32]:
y_predict.head()

860    negative
353    negative
333    negative
905    negative
289    negative
Name: review, dtype: object

In [33]:
metrics(y_predict, y_test)

Scores
---------------
Accuracy: 0.5125
Precision: 0.5109489051094891
Recall: 0.6965174129353234
F1: 0.5894736842105265


In [60]:
words = nltk.word_tokenize(X_test.reset_index(drop=True)[0])

In [63]:
clean_words = set(words).intersection(df_probas.columns)

In [66]:
probas = df_probas[list(clean_words)].prod(axis=1)

In [30]:
pred_class(X_test.reset_index(drop=True)[0], df_probas)

/home/kevcon/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:35: RuntimeWarning: overflow encountered in reduce
  return umr_prod(a, axis, dtype, out, keepdims)


'positive'

In [28]:
prediction = X_test.reset_index(drop=True).apply(lambda x: pred_class(x, df_probas))

In [47]:
metrics(prediction, y_test.reset_index(drop=True))

Scores
---------------
Accuracy: 0.4975
Precision: 0.4975
Recall: 1.0
F1: 0.664440734557596


In [40]:
print("Accuracy: %.3f"% accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction))

Accuracy: 0.497
             precision    recall  f1-score   support

   negative       0.00      0.00      0.00       201
   positive       0.50      1.00      0.66       199

avg / total       0.25      0.50      0.33       400



/home/kevcon/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Example

## Input

In [50]:
word_dict = {'text': 
             ["a great game", "the election was over", "very clean match", 
              "a clean but forgettable game", "it was a close election"],
            'tag': ["sports", "not_sports", "sports", "sports", "not_sports"]
            }

In [51]:
word_df = pd.DataFrame(word_dict)
word_df

,text,tag
0,a great game,sports
1,the election was over,not_sports
2,very clean match,sports
3,a clean but forgettable game,sports
4,it was a close election,not_sports


In [52]:
X = word_df.iloc[:, 0]
X

0                    a great game
1           the election was over
2                very clean match
3    a clean but forgettable game
4         it was a close election
Name: text, dtype: object

In [53]:
y = word_df.iloc[:, 1]
y

0        sports
1    not_sports
2        sports
3        sports
4    not_sports
Name: tag, dtype: object

In [62]:
X_test = pd.Series(['a very close game', "the election was over", 'game over match election'])

In [63]:
y_test = pd.Series(['sports', 'not_sports', 'not_sports'])

## Word Dataframe
Create sparse matrix of words in training dataset

In [64]:
df_words = create_word_mat(X, y)
df_words

,a,but,clean,close,election,forgettable,game,great,it,match,over,the,very,was,class_labels
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,sports
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,not_sports
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,sports
3,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sports
4,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,not_sports


## Word Probabilities
Calculate probabilities of each word per class label

In [77]:
df_probas = calc_probas(df_words)
df_probas

,a,but,clean,close,election,forgettable,game,great,it,match,over,the,very,was
class_labels,,,,,,,,,,,,,,
not_sports,-2.442347,-3.135494,-3.135494,-2.442347,-2.036882,-3.135494,-3.135494,-3.135494,-2.442347,-3.135494,-2.442347,-2.442347,-3.135494,-2.036882
sports,-2.120264,-2.525729,-2.120264,-3.218876,-3.218876,-2.525729,-2.120264,-2.525729,-3.218876,-2.525729,-3.218876,-3.218876,-2.525729,-3.218876


## Predictions
Predict class labels for test dataset

In [78]:
y_predict = X_test.apply(lambda x: pred_class(x, df_probas))
y_predict

0    not_sports
1        sports
2        sports
dtype: object

## Performance Metrics
Evaluate model performance based on:
- Accuracy
- Precision
- Recall

In [187]:
metrics(y_predict, y_test)

Scores
---------------
Accuracy: 0.3333333333333333
Precision: 1.0
Recall: 0.3333333333333333
F1: 0.5
